In [1]:
import csv
from collections import defaultdict

In [2]:
def count_conversations(file_path):
    # Structure: {language: {conversation_hash: count, 'total_unique': count, 'total_occurrences': count}}
    language_data = defaultdict(lambda: {'unique_conversations': set(), 'total_occurrences': 0})

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            conversation_hash = row['conversation_hash']
            language = row['language']

            # Track data per language
            lang_data = language_data[language]

            # Add to unique conversations if not already present
            if conversation_hash not in lang_data['unique_conversations']:
                lang_data['unique_conversations'].add(conversation_hash)

            # Increment total occurrences
            lang_data['total_occurrences'] += 1

    # Convert sets to counts and prepare final output
    result = {}
    for language, data in language_data.items():
        result[language] = {
            'unique_conversations': len(data['unique_conversations']),
            'total_occurrences': data['total_occurrences']
        }

    return result

In [3]:
file_path = 'codegrep_results_sql.csv'
language_conversations = count_conversations(file_path)

unique_convo_hash_good = 0
# Print results
print("Unique conversations and total occurrences per language:")
for language, data in language_conversations.items():
    print(f"Language: {language}")
    unique_convo_hash_good += data['unique_conversations']
    print(f"  Unique conversations: {data['unique_conversations']}")
    print(f"  Total occurrences: {data['total_occurrences']}")

Unique conversations and total occurrences per language:
Language: c
  Unique conversations: 18
  Total occurrences: 199
Language: csharp
  Unique conversations: 46
  Total occurrences: 156
Language: java
  Unique conversations: 82
  Total occurrences: 705
Language: javascript
  Unique conversations: 122
  Total occurrences: 616
Language: php
  Unique conversations: 102
  Total occurrences: 328
Language: python
  Unique conversations: 600
  Total occurrences: 5310


In [4]:
allowed_rules = {
    "java": [
        "tainted-sql-string",
        "tainted-sqli",
        "hibernate-sqli",
        "jdbc-sqli",
        "jdo-sqli",
        "jpa-sqli",
        "tainted-sql-from-http-request",
        "turbine-sqli",
        "vertx-sqli",
        "mongodb-nosqli",
        "tainted-sql-string",
    ],
    "csharp": [
        "csharp-sqli",
    ],
    "javascript": [
        "knex-sqli",
        "mysql-sqli",
        "pg-sqli",
        "sequelize-sqli",
        "tainted-sql-string",
        "node-knex-sqli",
        "node-mssql-sqli",
        "node-mysql-sqli",
        "node-postgres-sqli",
    ],
    "php": [
        "tainted-sql-string",
        "laravel-sql-injection",
        "wp-sql-injection-audit",
        "",
    ],
    "python": [
        "mysql-sqli",
        "psycopg-sqli",
        "pymssql-sqli",
        "pymysql-sqli",
        "sqlalchemy-sqli",
        "tainted-sql-string",
        "sql-injection-using-extra-where",
        "sql-injection-using-rawsql",
        "sql-injection-db-cursor-execute",
        "sql-injection-using-raw",
        "aiopg-sqli",
        "asyncpg-sqli",
        "pg8000-sqli",
        "pyramid-sqlalchemy-sql-injection",
        "sqlalchemy-sql-injection",
        "sqlalchemy-execute-raw-query",
        "avoid-sqlalchemy-text",
    ]
}

In [5]:
for key, value in allowed_rules.items():
    print(key, len(value))

java 11
csharp 1
javascript 9
php 4
python 17


In [6]:
# Flatten the allowed rules into a single list
allowed_rules_list = [rule for sublist in allowed_rules.values() for rule in sublist]
results = defaultdict(lambda: {"count": 0, "hashes": set()})
# Read the CSV file
with open('codegrep_results.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    for row in csv_reader:
        error_id = row['error_id'].split('.')[-1]  # Extract the last part of the error_id
        if error_id in allowed_rules_list:
            conversation_hash = row['conversation_hash']
            if error_id in results:
                # Update the count and add the hash to the set
                results[error_id]["count"] += 1
                results[error_id]["hashes"].add(conversation_hash)
            else:
                # Initialize a new entry
                results[error_id] = {"count": 1, "hashes": {conversation_hash}}         
            
print("Rules and their Occurrences (unique conversation hashes and total occurrences):")
for rule in results:
    count = results[rule]["count"]
    unique_hashes = results[rule]["hashes"]
    print(f"Rule: {rule}, Unique Conversation Hashes Count: {len(unique_hashes)}")



Rules and their Occurrences (unique conversation hashes and total occurrences):
Rule: csharp-sqli, Unique Conversation Hashes Count: 3
Rule: jdbc-sqli, Unique Conversation Hashes Count: 4
Rule: tainted-sql-string, Unique Conversation Hashes Count: 11
Rule: sqlalchemy-execute-raw-query, Unique Conversation Hashes Count: 42
Rule: psycopg-sqli, Unique Conversation Hashes Count: 5
Rule: sql-injection-db-cursor-execute, Unique Conversation Hashes Count: 3
Rule: avoid-sqlalchemy-text, Unique Conversation Hashes Count: 2


## Total statistics

In [7]:
unique_convo_hash_bad = 0
for rule in results:
    count = results[rule]["count"]
    unique_hashes = results[rule]["hashes"]
    unique_convo_hash_bad += len(unique_hashes)


In [8]:
print(unique_convo_hash_good, unique_convo_hash_bad)
print(unique_convo_hash_bad / unique_convo_hash_good)

970 70
0.07216494845360824


In [9]:
## Save the results in a CSV file

In [10]:
with open("results/sql_occurrences.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Rule", "Unique Hash Count", "Hashes"])
    for rule, data in results.items():
        writer.writerow([
            rule,
            len(data["hashes"]),
            ";".join(data["hashes"])
        ])